### Hate Speech Detection System Twitter

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
df=pd.read_csv('twitter2.csv')
df.head(2)

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...


In [5]:
df.shape

(24783, 7)

In [8]:
# column class is our 
df['class'].unique()

array([2, 1, 0], dtype=int64)

In [13]:
df['class']=df['class'].map({0: "Hate Speech", 1: "Offensive Language", 2: "No Hate and Offensive"})

In [15]:
df.head(3)

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,No Hate and Offensive,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,Offensive Language,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,Offensive Language,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...


In [16]:
df=df[['class','tweet']]
df.head(3)

,class,tweet
0,No Hate and Offensive,!!! RT @mayasolovely: As a woman you shouldn't...
1,Offensive Language,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,Offensive Language,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...


In [17]:
import re
import nltk
from nltk.corpus import stopwords
import string

In [18]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [19]:
stopword=set(stopwords.words('english'))

In [20]:
stemmer=nltk.SnowballStemmer('english')

In [21]:
def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text

In [22]:
df['tweet']=df['tweet'].apply(clean)
df.head(3)

C:\Users\iamvi\AppData\Local\Temp/ipykernel_3100/443780482.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet']=df['tweet'].apply(clean)


,class,tweet
0,No Hate and Offensive,rt mayasolov woman shouldnt complain clean ho...
1,Offensive Language,rt boy dat coldtyga dwn bad cuffin dat hoe ...
2,Offensive Language,rt urkindofbrand dawg rt ever fuck bitch sta...


In [27]:
X = np.array(df["tweet"])
y = np.array(df["class"])

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

In [29]:
X=cv.fit_transform(X)

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=41)

In [31]:
from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier()

In [32]:
model.fit(X_train,y_train)

DecisionTreeClassifier()

In [33]:
model.score(X_train,y_train)

0.9993005864313768

In [34]:
model.score(X_test,y_test)

0.8702388637830859

In [ ]:
def hatespeechfinder():
    import streamlit as st
    st.title('Hate Speech Detection')
    user_text=st.text_area('Enter any tweet:')
    if len(user_text)<1:
        st.write(" ")
    else:
        sample=user_text
        sample=cv.transform([sample]).toarray()
        x=model.predict(sample)
        st.title(x)
hatespeechfinder()